# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession

session = SparkSession.builder.getOrCreate()
session

In [3]:
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

#### Passing other options to spark session:
    
    

In [4]:
session = SparkSession.builder.config("someoption.key", "somevalue").getOrCreate()

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [5]:
import random

random.choice(['a','b','c'])

'b'

In [6]:
random.seed(42)

ids = range(12)

positions = [random.choice(['mechanic', 'sales', 'manager']) for id_ in ids]

positions

['manager',
 'mechanic',
 'mechanic',
 'manager',
 'sales',
 'mechanic',
 'mechanic',
 'mechanic',
 'manager',
 'mechanic',
 'manager',
 'manager']

In [7]:
rows = zip(ids, positions)

df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

In [8]:
df.collect()

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales'),
 Row(_1=5, _2='mechanic'),
 Row(_1=6, _2='mechanic'),
 Row(_1=7, _2='mechanic'),
 Row(_1=8, _2='manager'),
 Row(_1=9, _2='mechanic'),
 Row(_1=10, _2='manager'),
 Row(_1=11, _2='manager')]

In [9]:
df.take(5)

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

In [10]:
df.show(5)

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [11]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

In [12]:
rows = [Row(id=id_, position=position_) for id_, position_ in zip(ids, positions)]

rows

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales'),
 Row(id=5, position='mechanic'),
 Row(id=6, position='mechanic'),
 Row(id=7, position='mechanic'),
 Row(id=8, position='manager'),
 Row(id=9, position='mechanic'),
 Row(id=10, position='manager'),
 Row(id=11, position='manager')]

In [13]:
df  = session.createDataFrame(rows)

df.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [14]:
session.createDataFrame(zip(ids, positions), schema=['id', 'position']).show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [15]:
rdd = session.sparkContext.textFile('coupon150720.csv')

rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [16]:
split_rdd = rdd.map(lambda line:line.split(","))

split_rdd.take(2)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0']]

In [17]:
new_df = session.createDataFrame(split_rdd)
new_df.show(5)

+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|            _1| _2| _3| _4| _5| _6|    _7| _8| _9|_10|_11| _12|   _13|_14| _15|
+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|  H|0526|150904| OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|  H|6120|150905| OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|  H|2768|150721| OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|  S|0546|150804| OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|  V|0501|150803| OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
only showing top 5 rows



In [18]:
new_df

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

In [19]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Inferring and specifying schemas

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [20]:
from pyspark.sql import types

types.IntegerType()

IntegerType

In [21]:
fields = [types.StructField('id', types.IntegerType()),
          types.StructField('position', types.StringType())]

my_schema = types.StructType(fields)

my_schema

StructType(List(StructField(id,IntegerType,true),StructField(position,StringType,true)))

In [22]:
session.createDataFrame(zip(ids, positions), schema=my_schema)

DataFrame[id: int, position: string]

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [23]:
df_from_csv = session.read.csv('coupon150720.csv')

df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [24]:
df_with_sql = session.sql('SELECT * FROM csv.`coupon150720.csv`')

df_with_sql.show(2)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W|56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W|84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 2 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [25]:
session.read.json

<bound method DataFrameReader.json of <pyspark.sql.readwriter.DataFrameReader object at 0x7fc848cb1c50>>

### Basic operations with DataFrames

In [26]:
df.show(2)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
+---+--------+
only showing top 2 rows



In [27]:
df.take(2)

[Row(id=0, position='manager'), Row(id=1, position='mechanic')]

In [28]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [29]:
df.select(df['id'])

DataFrame[id: bigint]

In [30]:
df.select('id')

DataFrame[id: bigint]

In [31]:
filtered = df.filter(df['id'] > 5)
filtered

DataFrame[id: bigint, position: string]

In [32]:
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



`where` is exactly synonimous with `filter`

In [33]:
whered = df.where(df['id'] > 5)
whered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



#### Exercise

Extract all employee ids which correspond to managers

In [34]:
df.filter(df['position']=='manager')\
    .select(df['id']).show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [35]:
df.select(df['id'])\
    .where(df['position']=='manager').show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [36]:
df[df['position'] == 'manager'].select('id').show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [37]:
df_from_csv.schema.names

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14']

In [38]:
df_from_csv.select(['_c0', '_c1']).show()

+--------------+---+
|           _c0|_c1|
+--------------+---+
|79062005698500|  1|
|79062005698500|  2|
|79062005924069|  1|
|79065668570385|  1|
|79065668737021|  1|
|79062006192650|  1|
|79062006192650|  2|
|79062005733853|  1|
|79062005836987|  1|
|79062005836987|  2|
|79062005836987|  3|
|79062005836987|  4|
|79062005836987|  5|
|79062005836987|  6|
|79062005836987|  7|
|79062005862818|  1|
|79062005862818|  2|
|79065668498895|  1|
|79065668498895|  2|
|79062005534346|  1|
+--------------+---+
only showing top 20 rows



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we not assing to them. We need to create new RDDS with the appropriate columns.

In [39]:
df['anewcolumn'] = df['id']*100

TypeError: 'DataFrame' object does not support item assignment

In [ ]:
df2 = df.withColumn('anewcolumn', df['id'] * 100)

In [ ]:
df3 = df.select('id', 'position', df['id'] * 100)

df3.show()

### User defined functions

Creating `udf`s allows us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [ ]:
from pyspark.sql import functions

In [ ]:
df.select(functions.sqrt('id')).show()

In [ ]:
import math

In [ ]:
udf_log1p = functions.udf(math.log1p)

In [ ]:
df.printSchema()

In [ ]:
df3 = df.select('id', 'position', udf_log1p('id'))

df3.show()

In [ ]:
df3.printSchema()

In [ ]:
udf_log1p_typed = functions.udf(math.log1p, types.FloatType())

In [ ]:
df3 = df.select('id', 'position', udf_log1p_typed('id'))

df3.show()

If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [ ]:
df3.printSchema()

Think about this function: what is its return type?

In [ ]:
def odd_letters(word):
    return word[::2]

odd_letters('manager')

In [ ]:
odd_udf = functions.udf(odd_letters)

In [ ]:
df.select(odd_udf('position')).show()

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [ ]:
def salary(position):
    
    salaries = {'mechanic': 30000,
                'sales': 40000,
                'manager': 70000}
    
    return salaries.get(position)

print(salary('mechanic'))
print(salary('boss'))

In [ ]:
salary_udf = functions.udf(salary)

In [ ]:
df.select(salary_udf('position')).show()

In [ ]:
df_with_salaries = df.withColumn('salary', 
                                 salary_udf('position').alias('salary').cast(types.IntegerType()))

In [ ]:
df_with_salaries.show()

In [ ]:
df_with_salaries.printSchema()

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [ ]:
df_with_salaries.stat.corr('id', 'salary')

In [ ]:
df_with_salaries.stat.cov('id', 'salary')

### .crosstab()

In [ ]:
location_udf = functions.udf(lambda : random.choice(['Madrid', 'Barcelona']))

In [ ]:
df4 = df_with_salaries.withColumn('location', location_udf()).cache() # cache() for keeping in memory the evaluation

In [ ]:
df4.show()

In [ ]:
df4.crosstab('position', 'location').show()

### Grouping

In [ ]:
df4.groupby('location').mean().show()

In [ ]:
df4.groupby('location').mean('salary').show()

We can do each aggregation as in individual step, with a number of different syntaxes:

In [ ]:
df4.groupby('location').agg(functions.mean('salary').alias('average'), 
                            functions.stddev('salary').alias('std'))\
    .show()

In [ ]:
df4.groupby('location').agg({'salary': 'mean', 'id': 'mean'}).show()

### Intersections

Ver much like SQL joins.

In [ ]:
random.seed(42)
data = list(zip([10,11,9,10,9],
           [5000,10000,2000,0,1000],
           [random.choice(['Madrid', 'Barcelona', 'Sevila']) for _ in range(5)]
           ))
data

In [ ]:
new_df = session.createDataFrame(data, schema = ['id', 'Bonus', 'location'])

new_df.show()

In [ ]:
df4.join(new_df, on='id', how = 'left').show()

In [ ]:
help(df4.join)

In [ ]:
with_bonuses = df4.join(new_df, on=['id', 'location'], how = 'left')

In [ ]:
with_bonuses.show()

In [ ]:
df4.unpersist()

In [ ]:
sc.uiWebUrl

In [ ]:
df4.storageLevel

In [ ]:
from pyspark.storagelevel import StorageLevel

StorageLevel

In [ ]:
help(StorageLevel)

In [ ]:
df4.persist(StorageLevel.MEMORY_ONLY)

In [ ]:
df4.show()

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

In [ ]:
mean_std = df4.groupby('location').agg(functions.mean('salary').alias('mean'),\
                                     functions.stddev_pop('salary').alias('std'))

Or we can use the built-in functions defined in [the `pyspark.sql` module](http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions)

In [ ]:
df4.groupBy('location').mean('salary').show()

Once calculated, we annotate each employee with the values for their location and calculate their z-score

In [ ]:
df_joined = df4.join(mean_std, on = "location", how = 'left')

df_joined.show()

In [ ]:
df_joined.withColumn('z_value', (df_joined['salary'] - df_joined['mean']) / df_joined['std']).show()

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

In [ ]:
df4.join(mean_std, on=df4['location']==mean_std['location'], how = 'left').show()

### Handling null values

In [ ]:
the_nulls = session.createDataFrame([[101, 'manager', 120000, None],
                                     [None, None, 1500000, 'Haiti']])

the_nulls.show()

In [ ]:
df6 = df4.union(the_nulls)

df6.show()

In [ ]:
df6.dropna(subset='location').show()

In [ ]:
df6.dropna(thresh=3).show()

In [ ]:
df6.fillna('unknown').show()

In [ ]:
df6.fillna('unknown', subset = 'location').show()

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [ ]:
df6.registerTempTable('df6_table')

session.sql('SELECT * FROM df6_table WHERE location IS NULL').show()

In [ ]:
session.sql("""SELECT location, 
                      avg(salary) AS avg_salary, 
                      stddev_pop(salary) AS std_salary 
               FROM df6_table 
               GROUP BY location""").show()

In [ ]:
def zscore(value, mean, stddev):
    return (value-mean)/stddev

zscore_udf = functions.udf(zscore)

df_joined.select('id',
                 zscore_udf('salary', 'mean', 'std').alias('z-score')).show()

In [ ]:
df_joined.registerTempTable('df5_table')
session.udf.register('tocoto', zscore)

In [ ]:
session.sql('SELECT id, tocoto(salary, mean, std) AS z_score FROM df5_table').show()

Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [ ]:
pd_df = df_joined.toPandas()

pd_df.head(5)

In [ ]:
session.createDataFrame(pd_df).show()

## Writing out


In [ ]:
df_joined.write.csv('joined_data_frame', sep = "|")

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

In [ ]:
raw_coupons = session.read.csv('coupon150720.csv')

raw_coupons.show(5)

In [ ]:
df = raw_coupons.select(functions.col("_c0").alias("ticket_number"),
                        functions.col("_c1").alias("cupon_number").cast(types.IntegerType()),
                        functions.col("_c2").alias("origin"),
                        functions.col("_c3").alias("destination"),
                        functions.col("_c4").alias("airline"),
                        functions.col("_c6").alias("ticket_amount").cast(types.FloatType()))

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
raw_coupons.registerTempTable('raw_coupons')

coupons = session.sql("""SELECT 
                            CAST(_c0 AS LONG) AS tkt_number, 
                            CAST(_c1 AS INT) AS cpn_number,
                            _c2 AS origin,
                            _c3 AS destination,
                            _c4 AS airline,
                            CAST(_c6 AS FLOAT) AS amount
                        FROM raw_coupons""").show()

In [ ]:
df[df['destination'] == "MAD"]\
    .groupby('origin')\
    .agg(functions.sum('ticket_amount')).alias('ticket_amount_per_origin')\
    .show(10)

In [ ]:
airlines = df.filter(df['destination'] == "MAD")\
    .groupby('airline')\
    .mean('ticket_amount')

In [ ]:
airlines.sort('avg(ticket_amount)', ascending = False).show(10)

## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

